## Gated Field Network Autoencoder
### In  : Image & Posture
### Out : latent image/posture representation


In [1]:
import tensorflow as tf
import time
import scipy
import math 
import cv2
import cPickle as pickle
import random
import numpy as np
import matplotlib.pyplot as plt
import keras

from keras.models import Model
from keras.layers import Activation, Dense, Input, Multiply
from keras.layers import Conv2D, Flatten, Reshape, Conv2DTranspose
from keras.layers import Dot, Lambda, Concatenate, RepeatVector
from keras.utils import plot_model 
from PIL import Image
from keras.constraints import max_norm, non_neg 

Using TensorFlow backend.


In [2]:
pos_size = 3
img_size = 218
input_encoder_shape = (2, pos_size,)
latent_dim = 32
input_decoder_shape = (1, 2*latent_dim,)

In [11]:
def build_encoder(input_shape = input_encoder_shape):
    
    inputs = Input(shape = input_shape, name = 'encoder_input')
    
    x = Lambda(lambda x: x[:,0,:])(inputs)
    x = Reshape((1,pos_size,))(x)

    y = Lambda(lambda x: x[:,1,:])(inputs)
    y = Reshape((1,pos_size,))(y)

    fx = Flatten()(x)
    fx = Dense(latent_dim, activation = 'relu', name = 'latent_enc_fx1')(fx)
    fx = Dense(latent_dim, activation = 'relu', name = 'latent_enc_fx2')(fx)
    fx = Dense(latent_dim, activation = 'relu', name = 'latent_enc_fx3')(fx)
    fx = Reshape((latent_dim,1,))(fx)

    fy = Flatten()(y)
    fy = Dense(latent_dim, activation = 'relu', name = 'latent_enc_fy1')(fy)
    fy = Dense(latent_dim, activation = 'relu', name = 'latent_enc_fy2')(fy)
    fy = Dense(latent_dim, activation = 'relu', name = 'latent_enc_fy3')(fy)
    fy = Reshape((1,latent_dim,))(fy)

    matmul = Multiply()([fx, fy])

    fh = Flatten()(matmul)
    fh = Dense(latent_dim, name = 'latent_fh1')(fh)
    fh = Dense(latent_dim, name = 'latent_fh2')(fh)
    fh = Dense(latent_dim, name = 'latent_fh3')(fh)
    
    fx = Reshape((1, latent_dim,))(fx)
    fh = Reshape((1,latent_dim,))(fh)
    
    outputs = Concatenate()([fx, fh])
    encoder = Model(inputs = inputs, outputs = outputs, name = 'encoder_model')
    
    return encoder

def build_decoder(input_shape = input_decoder_shape):
    
    inputs = Input(shape = input_shape, name = 'decoder_input')
    
    fx = Lambda(lambda x: x[:,:,:latent_dim])(inputs)
    fx = Reshape((latent_dim,1,))(fx)
    
    fh = Lambda(lambda x: x[:,:,latent_dim:])(inputs)
        
    fh = Flatten()(fh)
    fh = Dense(latent_dim, name = 'latent_dec_fh1')(fh)
    fh = Dense(latent_dim, name = 'latent_dec_fh2')(fh)
    fh = Dense(latent_dim, name = 'latent_dec_fh3')(fh)
    fh = Reshape((1,latent_dim,))(fh)
                                              
    matmul = Multiply()([fx, fh])
    
    fy = Flatten()(matmul)
    fy = Dense(latent_dim, name = 'latent_dec_fy1')(fy)
    fy = Dense(latent_dim, name = 'latent_dec_fy2')(fy)
    fy = Dense(latent_dim, name = 'latent_dec_fy3')(fy)
    
    y = Dense(pos_size, activation = 'relu', name = 'y_recon')(fy)
    outputs = Reshape((1,pos_size,))(y)

    decoder = Model(inputs = inputs, outputs = outputs, name = 'decoder_model')
    
    return decoder
    
def build_autoencoder():
    encoder = build_encoder(input_shape = input_encoder_shape)
    decoder = build_decoder(input_shape = input_decoder_shape)
    
    autoencoder = Model(inputs = encoder.inputs, outputs = decoder(encoder(encoder.inputs)))
    
    return autoencoder

In [12]:
a = build_autoencoder()

In [13]:
a.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 2, 3)              0         
_________________________________________________________________
encoder_model (Model)        (None, 1, 64)             39392     
_________________________________________________________________
decoder_model (Model)        (None, 1, 3)              38179     
Total params: 77,571
Trainable params: 77,571
Non-trainable params: 0
_________________________________________________________________
